In [1]:
using BenchmarkTools

In [2]:
function Convolution_2d(input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    output = zeros(Float32, output_rows, output_columns)
    sumret = zeros(size(kernel))
    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            sumret .= patch .* kernel
            output[r, c] = sum(sumret) + bias
            sumret .= 0.0
        end
    end
    return output
end

function Convolution_2d!(ret, input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    sumret = zeros(size(kernel))
    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            sumret .= patch .* kernel
            ret[r, c] = sum(sumret) + bias
            sumret .= 0.0
        end
    end
end

function Convolution_2d_v2!(ret, sumret, input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    if padding
        padded_input = zeros(Float32, input_rows + 2*kernel_height - 2, input_columns + 2*kernel_width - 2)
        padded_input[kernel_height:end-kernel_height+1, kernel_width:end-kernel_width+1] .= input
        input_rows, input_columns = size(padded_input)
        input = padded_input
    end

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    for c in 1:output_columns
        for r in 1:output_rows
            patch = @view input[r:r+kernel_height-1, c:c+kernel_width-1]
            @views sumret .= patch .* kernel
            ret[r, c] = sum(sumret) + bias
            sumret .= 0.0
        end
    end
end

function Convolution_2d_v3!(ret, input, kernel; bias=0., padding=false)
    input_rows, input_columns = size(input)
    kernel_height, kernel_width = size(kernel)

    output_rows = input_rows - kernel_height + 1
    output_columns = input_columns - kernel_width + 1
    for c in 1:kernel_width
        for r in 1:kernel_height
            @views ret .+= input[r:r+input_rows-kernel_height, c:c+input_columns-kernel_width] .* kernel[r,c]
        end
    end
    ret .+= bias
end

Convolution_2d_v3! (generic function with 1 method)

In [3]:
function Conv_forward_v1(input, weights, bias)
    input_height, input_width, input_channels = size(input)
    kernel_height, kernel_width, _, output_channels = size(weights)

    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1
    output = zeros(Float32, output_height, output_width, output_channels)

    for k in 1:output_channels
        for c in 1:input_channels
            output[:, :, k] += Convolution_2d(input[:, :, c], weights[:, :, c, k]; bias=bias[k])
        end
    end
    
    return output
end

Conv_forward_v1 (generic function with 1 method)

In [4]:
function Conv_forward_v2(input, weights, bias)
    input_height, input_width, input_channels = size(input)
    kernel_height, kernel_width, _, output_channels = size(weights)

    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1
    output = zeros(Float32, output_height, output_width, output_channels)

    for k in 1:output_channels
        for c in 1:input_channels
            output[:, :, k] .+= Convolution_2d(input[:, :, c], weights[:, :, c, k]; bias=bias[k])
        end
    end
    
    return output
end

Conv_forward_v2 (generic function with 1 method)

In [5]:
function Conv_forward_v3(input, weights, bias)
    input_height, input_width, input_channels = size(input)
    kernel_height, kernel_width, _, output_channels = size(weights)

    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1
    output = zeros(Float32, output_height, output_width, output_channels)
    ret = zeros(Float32, output_height, output_width)

    for k in 1:output_channels
        for c in 1:input_channels
            Convolution_2d!(ret, input[:, :, c], weights[:, :, c, k]; bias=bias[k])
            output[:, :, k] .+= ret
            ret .= 0.0
        end
    end
    
    return output
end

Conv_forward_v3 (generic function with 1 method)

In [6]:
function Conv_forward_v4(input, weights, bias)
    input_height, input_width, input_channels = size(input)
    kernel_height, kernel_width, _, output_channels = size(weights)

    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1
    output = zeros(Float32, output_height, output_width, output_channels)
    ret = zeros(Float32, output_height, output_width)
    tmp_input = zeros(Float32, input_height, input_width)
    tmp_weights = zeros(Float32, kernel_height, kernel_width)
    for k in 1:output_channels
        for c in 1:input_channels
            tmp_input .= @views input[:, :, c]
            tmp_weights .= @views weights[:, :, c, k]
            Convolution_2d!(ret, tmp_input, tmp_weights; bias=bias[k])
            output[:, :, k] .+= ret
            ret .= 0.0
        end
    end
    
    return output
end

Conv_forward_v4 (generic function with 1 method)

In [7]:
function Conv_forward_v5(input, weights, bias)
  input_height, input_width, input_channels = size(input)
  kernel_height, kernel_width, _, output_channels = size(weights)

  output_height = input_height - kernel_height + 1
  output_width = input_width - kernel_width + 1
  output = zeros(Float32, output_height, output_width, output_channels)
  ret = zeros(Float32, output_height, output_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  for k in 1:output_channels
      for c in 1:input_channels
          tmp_input .= @views input[:, :, c]
          tmp_weights .= @views weights[:, :, c, k]
          Convolution_2d!(ret, tmp_input, tmp_weights; bias=bias[k])
          @views output[:, :, k] .+= ret
          ret .= 0.0
      end
  end
  return output
end

Conv_forward_v5 (generic function with 1 method)

In [8]:
function Conv_forward_v6(input, weights, bias)
  input_height, input_width, input_channels = size(input)
  kernel_height, kernel_width, _, output_channels = size(weights)

  output_height = input_height - kernel_height + 1
  output_width = input_width - kernel_width + 1
  output = zeros(Float32, output_height, output_width, output_channels)
  ret = zeros(Float32, output_height, output_width)
  sumret = zeros(Float32, kernel_height, kernel_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  
  for k in 1:output_channels
      for c in 1:input_channels
          tmp_input .= @views input[:, :, c]
          tmp_weights .= @views weights[:, :, c, k]
          Convolution_2d_v2!(ret, sumret, tmp_input, tmp_weights; bias=bias[k])
          @views output[:, :, k] .+= ret
          ret .= 0.0
      end
  end
  return output
end

Conv_forward_v6 (generic function with 1 method)

In [9]:
function Conv_forward_v7(input, weights, bias)
  input_height, input_width, input_channels = size(input)
  kernel_height, kernel_width, _, output_channels = size(weights)

  output_height = input_height - kernel_height + 1
  output_width = input_width - kernel_width + 1
  output = zeros(Float32, output_height, output_width, output_channels)
  ret = zeros(Float32, output_height, output_width)
  tmp_input = zeros(Float32, input_height, input_width)
  tmp_weights = zeros(Float32, kernel_height, kernel_width)
  
  for k in 1:output_channels
      for c in 1:input_channels
          tmp_input .= @views input[:, :, c]
          tmp_weights .= @views weights[:, :, c, k]
          Convolution_2d_v3!(ret, tmp_input, tmp_weights; bias=bias[k])
          @views output[:, :, k] .+= ret
          ret .= 0.0
      end
  end
  return output
end

Conv_forward_v7 (generic function with 1 method)

In [10]:
input = rand(Float32, 28, 28, 6);
weights = rand(Float32, 3, 3, 6, 16);
bias = rand(Float32, 16);

In [11]:
@benchmark Conv_forward_v1(input, weights, bias)

BenchmarkTools.Trial: 1406 samples with 1 evaluation.
 Range (min … max):  2.991 ms …  11.325 ms  ┊ GC (min … max): 0.00% … 73.26%
 Time  (median):     3.390 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.545 ms ± 937.936 μs  ┊ GC (mean ± σ):  4.48% ± 10.54%

  ▂ ▁█▇▄▁                                                      
  ███████▃▅▅▃▃▁▅▁▁▁▁▃▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▆▇ █
  2.99 ms      Histogram: log(frequency) by time      9.04 ms <

 Memory estimate: 1.15 MiB, allocs estimate: 578.

In [12]:
@benchmark Conv_forward_v2(input, weights, bias)

BenchmarkTools.Trial: 1508 samples with 1 evaluation.
 Range (min … max):  2.909 ms …   9.160 ms  ┊ GC (min … max): 0.00% … 66.91%
 Time  (median):     3.213 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.312 ms ± 759.818 μs  ┊ GC (mean ± σ):  3.18% ±  8.74%

  ▆▁██▅▁                                                       
  ██████▆▄▆▆▁▄▄▄▄▄▄▁▁▄▁▄▁▁▁▁▄▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▅▇ █
  2.91 ms      Histogram: log(frequency) by time      8.69 ms <

 Memory estimate: 909.31 KiB, allocs estimate: 482.

In [13]:
@benchmark Conv_forward_v3(input, weights, bias)

BenchmarkTools.Trial: 1686 samples with 1 evaluation.
 Range (min … max):  2.872 ms …   8.691 ms  ┊ GC (min … max): 0.00% … 65.52%
 Time  (median):     2.899 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.964 ms ± 316.215 μs  ┊ GC (mean ± σ):  1.05% ±  4.86%

  █▆▄▂  ▄  ▁                                                   
  ████▆█████▇▆▃▄▃▁▁▁▄▃▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▆▃ █
  2.87 ms      Histogram: log(frequency) by time      4.63 ms <

 Memory estimate: 642.12 KiB, allocs estimate: 387.

In [14]:
@benchmark Conv_forward_v4(input, weights, bias)

BenchmarkTools.Trial: 1718 samples with 1 evaluation.
 Range (min … max):  2.833 ms …   8.391 ms  ┊ GC (min … max): 0.00% … 65.83%
 Time  (median):     2.866 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.907 ms ± 213.905 μs  ┊ GC (mean ± σ):  0.50% ±  3.39%

  ▅█▁                                                          
  ███▅▇▇▆▄▃▄▃▃▃▃▃▃▂▂▂▂▃▂▂▂▂▂▂▁▂▂▁▂▂▁▂▂▁▁▁▁▁▁▂▁▁▂▁▁▁▂▁▁▁▂▁▁▂▁▂ ▃
  2.83 ms         Histogram: frequency by time        3.44 ms <

 Memory estimate: 330.41 KiB, allocs estimate: 197.

In [15]:
@benchmark Conv_forward_v5(input, weights, bias)

BenchmarkTools.Trial: 1732 samples with 1 evaluation.
 Range (min … max):  2.792 ms …   5.652 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.826 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.885 ms ± 284.097 μs  ┊ GC (mean ± σ):  0.03% ± 0.69%

  █▆▆▃▁▁                                                       
  ████████▇▇▇▆▅▄▁▁▄▃▄▅▁▄▁▁▁▃▄▃▁▃▁▁▁▁▅▃▁▁▃▁▃▃▃▃▃▄▁▁▁▁▃▁▁▁▃▁▅▁▃ █
  2.79 ms      Histogram: log(frequency) by time      4.53 ms <

 Memory estimate: 60.41 KiB, allocs estimate: 101.

In [16]:
@benchmark Conv_forward_v6(input, weights, bias)

BenchmarkTools.Trial: 2025 samples with 1 evaluation.
 Range (min … max):  2.402 ms …   5.250 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.434 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.467 ms ± 121.326 μs  ┊ GC (mean ± σ):  0.05% ± 1.06%

   █▁▅                                                         
  ▅████▆▅▄▄▄▄▄▄▃▃▄▃▄▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▂▂▂▂▂▁▁▂▂▁▂▂▁▂▂▂ ▃
  2.4 ms          Histogram: frequency by time         2.8 ms <

 Memory estimate: 48.50 KiB, allocs estimate: 6.

In [17]:
@benchmark Conv_forward_v7(input, weights, bias)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  325.900 μs …  4.470 ms  ┊ GC (min … max): 0.00% … 91.70%
 Time  (median):     335.000 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   341.349 μs ± 60.977 μs  ┊ GC (mean ± σ):  0.48% ±  2.52%

   ▅█  ▄▇                                                       
  ▂███▅███▆▄▄▄▅▇▆▅▅▆▅▄▃▃▃▃▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂ ▃
  326 μs          Histogram: frequency by time          404 μs <

 Memory estimate: 48.41 KiB, allocs estimate: 5.

In [18]:
Conv_forward_v5(input, weights, bias)[:,:,7]

26×26 Matrix{Float32}:
 16.8845  18.2594  18.7286  18.822   …  16.9721  17.3893  18.4491  18.3908
 18.5714  16.517   18.1688  18.5788     17.277   18.1204  16.7065  16.8356
 18.7299  16.5257  17.0319  16.8556     17.6947  17.9055  16.5244  18.2157
 20.9102  19.3535  17.7175  15.0227     16.8224  16.8294  17.7     18.2396
 20.6838  20.5951  18.2939  15.821      16.9613  16.0222  17.1757  18.1366
 18.3121  19.088   17.2587  16.4646  …  16.5458  16.5134  15.3376  15.3293
 17.4604  19.5499  18.1163  16.7063     16.4121  15.2534  15.1732  17.5077
 16.6046  17.7405  18.2582  18.7267     16.9517  15.2968  17.4078  17.7513
 15.7168  14.7998  17.5295  18.6343     17.6783  16.0969  17.5986  17.7251
 17.1229  16.8894  16.3873  17.3629     17.0878  16.8594  17.6072  17.9415
 19.4259  18.1652  18.3607  18.1661  …  17.6095  19.1117  18.0503  18.7448
 18.962   19.97    18.8933  17.9475     17.985   19.0002  19.2493  18.8378
 18.5388  18.0807  17.5033  17.8047     19.0425  18.8824  18.1337  17.3338
 1

In [22]:
Conv_forward_v7(input, weights, bias)[:,:,7]

26×26 Matrix{Float32}:
 16.8845  18.2594  18.7286  18.822   …  16.972   17.3893  18.4491  18.3908
 18.5714  16.517   18.1688  18.5788     17.277   18.1204  16.7065  16.8356
 18.7299  16.5257  17.0319  16.8556     17.6947  17.9055  16.5244  18.2157
 20.9102  19.3535  17.7175  15.0227     16.8224  16.8294  17.7     18.2396
 20.6838  20.5951  18.2939  15.821      16.9613  16.0222  17.1757  18.1366
 18.3121  19.088   17.2587  16.4646  …  16.5458  16.5134  15.3376  15.3293
 17.4604  19.5499  18.1163  16.7063     16.4121  15.2534  15.1732  17.5077
 16.6046  17.7405  18.2582  18.7267     16.9517  15.2968  17.4078  17.7513
 15.7168  14.7998  17.5295  18.6343     17.6783  16.0969  17.5986  17.7251
 17.1229  16.8894  16.3873  17.3629     17.0878  16.8594  17.6072  17.9415
 19.4259  18.1652  18.3607  18.1661  …  17.6095  19.1117  18.0503  18.7448
 18.962   19.97    18.8933  17.9475     17.985   19.0002  19.2493  18.8378
 18.5388  18.0807  17.5033  17.8047     19.0425  18.8824  18.1337  17.3338
 1